In [41]:
import os
import pandas as pd
import numpy as np

import random

import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_palette("colorblind")
sns.set(rc={'figure.figsize':(7, 7)})

pd.set_option('display.max_columns', None)
pd.set_option("display.float_format", lambda x: f'{x:.2f}')

import warnings
warnings.filterwarnings("ignore")

In [2]:
def std_col_names(df):
    """
    - Convert feature names to lower case
    """
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    return df


In [3]:
os.listdir('./data/')

['sample_submission.csv', 'train.csv', 'test.csv']

In [4]:
# Load DF
train_df = std_col_names(pd.read_csv('./data/train.csv'))
df = train_df.copy()
df.head(5)

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,music_genre
0,MSC_83537.0,Estrellitas y Duendes,49.18,0.97,0.58,214625.78,0.19,0.00,Scale E,0.15,-14.14,NaN,Major,0.05,143.78799999999998,Band,0.60,Jazz
1,MSC_22044.0,Al Norte,59.83,1.01,0.69,216232.20,0.27,0.00,Scale A,0.17,-13.72,Male,?,0.04,?,Band,0.36,Jazz
2,MSC_62017.0,Yeah! (feat. Lil Jon & Ludacris),89.02,0.02,0.97,273314.72,0.86,0.00,Scale D,0.04,-5.00,Female,Major,0.12,105.01799999999999,Band,0.64,Rap
3,MSC_76365.0,Can’t You See,55.76,0.01,0.62,189189.61,0.97,0.00,Scale D,0.12,-4.26,Both,Major,0.17,?,Duet,0.98,Rock
4,MSC_71493.0,"Sonata III (G Moll), BWV 1029: Adagio",45.09,0.86,0.28,410136.99,0.11,0.00,Scale F Sharp,0.13,-26.92,Female,Minor,0.04,112.18299999999999,Band,0.07,Classical


In [5]:
# Observe missing data and feature types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15681 entries, 0 to 15680
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        15681 non-null  object 
 1   track_name         15681 non-null  object 
 2   popularity         15039 non-null  float64
 3   acousticness       15681 non-null  float64
 4   danceability       15125 non-null  float64
 5   duration_ms        15587 non-null  float64
 6   energy             15587 non-null  float64
 7   instrumentalness   15586 non-null  float64
 8   key                15681 non-null  object 
 9   liveness           15681 non-null  float64
 10  loudness           15645 non-null  float64
 11  voice_gender       14916 non-null  object 
 12  mode               15646 non-null  object 
 13  speechiness        15655 non-null  float64
 14  tempo              15681 non-null  object 
 15  musician_category  14321 non-null  object 
 16  valence            150

In [6]:
# Find out missing column names
missing_cols = df.columns[df.isnull().any()].to_list()
print('Features with missing values: ')
missing_cols

Features with missing values: 


['popularity',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'loudness',
 'voice_gender',
 'mode',
 'speechiness',
 'musician_category',
 'valence']

In [7]:
# Evaluate count of missing values
missing_cols_count = df[missing_cols].isnull().sum()
print('Number of missing values for each feature in descending order: ')
(missing_cols_count)#.sort_values(ascending= False)

Number of missing values for each feature in descending order: 


popularity            642
danceability          556
duration_ms            94
energy                 94
instrumentalness       95
loudness               36
voice_gender          765
mode                   35
speechiness            26
musician_category    1360
valence               677
dtype: int64

In [8]:
# View data distributions
print('Descriptive stats of numerical features: ')
df.describe().loc[['min', 'max', 'mean', '50%']]

Descriptive stats of numerical features: 


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,valence
min,0.00,0.00,0.06,-1.00,-1.00,0.00,0.02,-50.05,0.02,0.00
max,108.51,1.09,1.07,3410383.32,1.09,1.08,1.08,1.39,1.00,1.08
mean,50.82,0.45,0.57,248000.81,0.43,0.26,0.19,-11.62,0.10,0.45
50%,54.11,0.30,0.59,236212.74,0.56,0.00,0.13,-8.61,0.05,0.43


In [9]:
# View data distributions
print('Descriptive stats of numerical features: ')
df.describe(include= 'object')

Descriptive stats of numerical features: 


,instance_id,track_name,key,voice_gender,mode,tempo,musician_category,music_genre
count,15681,15681,15681,14916,15646,15681,14321,15681
unique,15681,14799,12,3,3,12332,3,7
top,MSC_83537.0,Smile,Scale C,Female,Major,?,Band,Classical
freq,1,7,1771,5050,9210,1476,4843,3990


In [10]:
print('Unique values in each feature: ')
df.nunique()

Unique values in each feature: 


instance_id          15681
track_name           14799
popularity           12743
acousticness         15681
danceability         15125
duration_ms          14011
energy               14371
instrumentalness     11219
key                     12
liveness             15681
loudness             10473
voice_gender             3
mode                     3
speechiness          15655
tempo                12332
musician_category        3
valence              15004
music_genre              7
dtype: int64

In [11]:
print('Unique music_genres: ')
print(df['music_genre'].value_counts())

Unique music_genres: 
Classical     3990
Rock          3879
Rap           3207
Jazz          2850
Country        683
Electronic     614
Hip-Hop        458
Name: music_genre, dtype: int64


In [12]:
print(f'Number of records with duration= -1 :')
print(df[df['duration_ms']==-1].shape[0])

print(f'Number of records with duration= np.nan :')
print(df['duration_ms'].isna().sum())

Number of records with duration= -1 :
1577
Number of records with duration= np.nan :
94


In [13]:
print(f'Number of records with energy= -1 :')
print(df[df['energy']==-1].shape[0])

print(f'Number of records with duration= np.nan :')
print(df['energy'].isna().sum())

Number of records with energy= -1 :
1217
Number of records with duration= np.nan :
94


In [14]:
print('Loudness value count greater than 0: ')
print(df[df['loudness']>=0].shape[0])

Loudness value count greater than 0: 
7


In [15]:
df.head()

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,music_genre
0,MSC_83537.0,Estrellitas y Duendes,49.18,0.97,0.58,214625.78,0.19,0.00,Scale E,0.15,-14.14,NaN,Major,0.05,143.78799999999998,Band,0.60,Jazz
1,MSC_22044.0,Al Norte,59.83,1.01,0.69,216232.20,0.27,0.00,Scale A,0.17,-13.72,Male,?,0.04,?,Band,0.36,Jazz
2,MSC_62017.0,Yeah! (feat. Lil Jon & Ludacris),89.02,0.02,0.97,273314.72,0.86,0.00,Scale D,0.04,-5.00,Female,Major,0.12,105.01799999999999,Band,0.64,Rap
3,MSC_76365.0,Can’t You See,55.76,0.01,0.62,189189.61,0.97,0.00,Scale D,0.12,-4.26,Both,Major,0.17,?,Duet,0.98,Rock
4,MSC_71493.0,"Sonata III (G Moll), BWV 1029: Adagio",45.09,0.86,0.28,410136.99,0.11,0.00,Scale F Sharp,0.13,-26.92,Female,Minor,0.04,112.18299999999999,Band,0.07,Classical


In [16]:
print('Unqiue key values: ')
print(df['key'].unique())

Unqiue key values: 
['Scale E' 'Scale A' 'Scale D' 'Scale F Sharp' 'Scale G Sharp'
 'Scale C Sharp' 'Scale A Sharp' 'Scale F' 'Scale G' 'Scale B' 'Scale C'
 'Scale D Sharp']


In [17]:
print('Unique mode values: ')
print(df['mode'].value_counts())


Unique mode values: 
Major    9210
Minor    5357
?        1079
Name: mode, dtype: int64


In [18]:
print('Tempo records with `?`: ')
print(df[df['tempo']=='?'].shape[0])

Tempo records with `?`: 
1476


In [19]:
df[df['tempo']=='?']['music_genre'].value_counts()

Classical     396
Rock          328
Rap           325
Jazz          276
Electronic     58
Country        56
Hip-Hop        37
Name: music_genre, dtype: int64

In [20]:
print('Value counts of musician_category')
print(df['musician_category'].value_counts())

df[['track_name', 'musician_category']]

Value counts of musician_category
Band    4843
Solo    4784
Duet    4694
Name: musician_category, dtype: int64


,track_name,musician_category
0,Estrellitas y Duendes,Band
1,Al Norte,Band
2,Yeah! (feat. Lil Jon & Ludacris),Band
3,Can’t You See,Duet
4,"Sonata III (G Moll), BWV 1029: Adagio",Band
...,...,...
15676,I. Andante,Band
15677,Only Trust Your Heart,Duet
15678,Moment's Notice,NaN
15679,Such Small Scenes,Duet


In [21]:
df.columns

Index(['instance_id', 'track_name', 'popularity', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'voice_gender', 'mode', 'speechiness', 'tempo',
       'musician_category', 'valence', 'music_genre'],
      dtype='object')

In [22]:
df['tempo'].replace('?', np.nan, inplace= True)

df['tempo'].fillna(0, inplace=True)
df['tempo'] = df['tempo'].astype('float')

In [80]:
genre_mean = df[['danceability', 'energy', 'loudness', 'tempo', 'valence', 'music_genre']].groupby('music_genre').mean().reset_index()
dance_dict = dict(zip(genre_mean['music_genre'], genre_mean['danceability']))

In [50]:
duration_mean = df['duration_ms'].mean()
df['duration_ms'].fillna(duration_mean, inplace=True)
df['duration_mins'] = df['duration_ms'] / 60000

instrumentalness_mean = df['instrumentalness'].mean()
df['instrumentalness'].fillna(instrumentalness_mean, inplace= True)

speechiness_mean = df['speechiness'].mean()
df['speechiness'].fillna(speechiness_mean, inplace= True)

popularity_mean = df['popularity'].mean()
df['popularity'].fillna(popularity_mean, inplace= True)

In [26]:
df['voice_gender'].fillna('No', inplace= True)

df['mode'].replace('?', np.nan, inplace= True)

In [40]:
df['mode'].fillna(random.choice(['Major', 'Minor']), inplace= True)

df['musician_category'].fillna(random.choice(['Band', 'Duet', 'Solo']), inplace= True)

In [107]:
def fill_by_genre_mean(df, features):
    for feature in features:
        df[feature].fillna(df.groupby('music_genre')[feature].transform('mean'), inplace= True)
    return df

df = fill_by_genre_mean(df,['danceability', 'energy', 'loudness', 'tempo', 'valence'] )

In [108]:
fin_missing_cols = df.columns[df.isnull().any()].to_list()
fin_missing_cols_count = df[fin_missing_cols].isnull().sum()
fin_missing_cols_count

Series([], dtype: float64)

## References
- [Replace invalid values](https://stackoverflow.com/questions/17097236/replace-invalid-values-with-none-in-pandas-dataframe)
- [Replace values randomly](https://stackoverflow.com/questions/36413314/filling-missing-data-by-random-choosing-from-non-missing-values-in-pandas-datafr)
- [Pandas: filling missing values by mean in each group](https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group)